
## Downloading the data 

### Elevation

TODO adapt from swan coastal

```bash
mkdir -p $HOME/data/Lithology/swan_coastal
cd $HOME/data/Lithology/swan_coastal
curl -o Swan_DEM.7z https://cloudstor.aarnet.edu.au/plus/s/HL3h7u67xUA1ilR/download
7za x Swan_DEM.7z
```

Windows:

```bat
mkdir -p c:\data\Lithology\swan_coastal
cd c:\data\Lithology\swan_coastal
curl -o Swan_DEM.7z https://cloudstor.aarnet.edu.au/plus/s/HL3h7u67xUA1ilR/download
7z x Swan_DEM.7z
```

### Borehole logs

TODO adapt from swan coastal

```bash
mkdir -p $HOME/data/Lithology/
cd $HOME/data/Lithology/
curl -o NGIS_SCP.7z https://cloudstor.aarnet.edu.au/plus/s/KLEBWonjOAOPVVd/download
7za x NGIS_SCP.7z
```

Windows:

```bat
cd c:\data\Lithology
curl -o NGIS_SCP.7z https://cloudstor.aarnet.edu.au/plus/s/KLEBWonjOAOPVVd/download
7z x NGIS_SCP.7z
```



In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
import geopandas as gpd


In [ ]:
# Only set to True for co-dev of ela from this use case:
ela_from_source = False
ela_from_source = True

In [ ]:
if ela_from_source:
    if ('ELA_SRC' in os.environ):
        root_src_dir = os.environ['ELA_SRC']
    elif sys.platform == 'win32':
        root_src_dir = r'C:\src\github_jm\pyela'
    else:
        username = os.environ['USER']
        root_src_dir = os.path.join('/home', username, 'src/ela/pyela')
    pkg_src_dir = root_src_dir
    sys.path.insert(0, pkg_src_dir)

from ela.textproc import *
from ela.utils import *
from ela.classification import *
from ela.visual import *
from ela.spatial import SliceOperation

## Importing data

There are two main sets of information we need: the borehole lithology logs, and the spatial information in the surface elevation (DEM) and geolocation of a subset of bores around Bungendore. 

In [ ]:
data_path = None

You probably want to explicitly set `data_path` to the location where you put the folder(s) e.g:

In [ ]:
#data_path = '/home/myusername/data' # On Linux, if you now have the folder /home/myusername/data/Bungendore
#data_path = r'C:\data\Lithology'  # windows, if you have C:\data\Lithology\Bungendore

Otherwise a fallback for the pyela developer(s)

In [ ]:
if data_path is None:
    if ('ELA_DATA' in os.environ):
        data_path = os.environ['ELA_DATA']
    elif sys.platform == 'win32':
        data_path = r'C:\data\Lithology'
    else:
        username = os.environ['USER']
        data_path = os.path.join('/home', username, 'data')

In [ ]:
data_path

In [ ]:
#aem_datadir = os.path.join(data_path, 'AEM')
cbr_datadir = os.path.join(data_path, 'Canberra')
cbr_datadir_out = os.path.join(cbr_datadir, 'out')
# scp_datadir = os.path.join(aem_datadir, 'Swan_coastal_plains')
# scp_grids_datadir = os.path.join(scp_datadir, 'grids')
ngis_datadir = os.path.join(data_path, 'NGIS')
act_shp_datadir = os.path.join(ngis_datadir, 'shp_ACT')
bidgee_shp_datadir = os.path.join(ngis_datadir, 'shp_murrumbidgee_river')

## DEM


In [ ]:
dem = rasterio.open(os.path.join(cbr_datadir,'CLIP.tif'))

In [ ]:
dem.crs

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
show(dem,title='Canberra', cmap='terrain',  ax=ax)

## Bore data

In [ ]:
bore_locations_raw = gpd.read_file(os.path.join(cbr_datadir, 'Bores/act_bores.shp'))

In [ ]:
bore_locations_raw.columns

In [ ]:
bore_locations_raw.crs, dem.crs

The DEM raster and the bore location shapefile do not use the same projection (coordinate reference system) so we reproject one of them. We choose the raster's UTM.

In [ ]:
bore_locations = bore_locations_raw.to_crs(dem.crs)

In [ ]:
lithology_logs_act = pd.read_csv(os.path.join(act_shp_datadir, 'NGIS_LithologyLog.csv'))
lithology_logs_bidgee = pd.read_csv(os.path.join(bidgee_shp_datadir, 'NGIS_LithologyLog.csv'))

In [ ]:
len(lithology_logs_act), len(lithology_logs_bidgee)

In [ ]:
lithology_logs = pd.concat([lithology_logs_act, lithology_logs_bidgee])

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
show(dem,title='Swan Coastal Plain', cmap='terrain',  ax=ax)
bore_locations.plot(ax=ax, facecolor='black')

### Subset to the location of interest

The lithology logs are for all of western australia, which is much larger than the area of interest and for which we have the geolocation of boreholes. We subset to the location of interest 

In [ ]:
df = lithology_logs.copy()

Let's subset the logs based on spatial locations, to keep only those "nearby" the DEM we have near the locality of Bungendore in this case study. First define a few constants: 

In [ ]:
df.columns

In [ ]:
DEPTH_FROM_COL = 'FromDepth'
DEPTH_TO_COL = 'ToDepth'

TOP_ELEV_COL = 'TopElev'
BOTTOM_ELEV_COL = 'BottomElev'

LITHO_DESC_COL = 'Description'
HYDRO_CODE_COL = 'HydroCode'

### Merging the geolocation from the shapefile and lithology records

The geopandas data frame has a column geometry listing `POINT` objects. 'ela' includes  `get_coords_from_gpd_shape` to extrace the coordinates to a simpler structure. 'ela' has predefined column names (e.g. EASTING_COL) defined for easting/northing information, that we can use to name our coordinate information.

In [ ]:
bore_locations.columns

In [ ]:
geoloc = get_coords_from_gpd_shape(bore_locations, colname='geometry', out_colnames=[EASTING_COL, NORTHING_COL])
geoloc[HYDRO_CODE_COL] = bore_locations[HYDRO_CODE_COL]
# We keep lat/long for potential leaflet viewer
geoloc['Latitude'] = bore_locations['Latitude']
geoloc['Longitude'] = bore_locations['Longitude']
geoloc.head()

In [ ]:
df.head()

With this data frame we can perform two operations in one go: subsetting the lithology records to only the 640 bores of interest, and adding to the result the x/y geolocations to the data frame.

In [ ]:
len(geoloc), len(df)

In [ ]:
geoloc[HYDRO_CODE_COL].dtype, df[HYDRO_CODE_COL].dtype

In [ ]:
# geoloc[HYDRO_CODE_COL] = geoloc[HYDRO_CODE_COL].apply(parse_int)
# df[HYDRO_CODE_COL] = df[HYDRO_CODE_COL].apply(parse_int)

In [ ]:
df = pd.merge(df, geoloc, how='inner', on=HYDRO_CODE_COL, sort=False, copy=True, indicator=False, validate=None)

In [ ]:
len(df)

In [ ]:
df.head()

### Round up 'depth to' and 'depth from' columns

We round the depth related columns to the upper integer value and drop the entries where the resulting depths have degenerated to 0. `ela` has a class `DepthsRounding` to facilitate this operations on lithology records with varying column names.

We first clean up height/depths columns to make sure they are numeric.

In [ ]:
def as_numeric(x):
    if isinstance(x, float):
        return x
    if x == 'None':
        return np.nan
    elif x is None:
        return np.nan
    elif isinstance(x, str):
        return float(x)
    else:
        return float(x)

In [ ]:
df[DEPTH_FROM_COL] = df[DEPTH_FROM_COL].apply(as_numeric)
df[DEPTH_TO_COL] = df[DEPTH_TO_COL].apply(as_numeric)
df[TOP_ELEV_COL] = df[TOP_ELEV_COL].apply(as_numeric)
df[BOTTOM_ELEV_COL] = df[BOTTOM_ELEV_COL].apply(as_numeric)

In [ ]:
dr = DepthsRounding(DEPTH_FROM_COL, DEPTH_TO_COL)

In [ ]:
"Before rounding heights we have " + str(len(df)) + " records"

In [ ]:
df = dr.round_to_metre_depths(df, np.round, True)
"After removing thin sliced entries of less than a metre, we are left with " + str(len(df)) + " records left"

## Exploring the descriptive lithology 

In [ ]:
descs = df[LITHO_DESC_COL]
descs = descs.reset_index()
descs = descs[LITHO_DESC_COL]
descs.head()

The description column as read seems to be objects. Other columns seem to be objects when they should be numeric. We define two functions to clean these.

In [ ]:
def clean_desc(x):
    if isinstance(x, float):
        return u''
    elif x is None:
        return u''
    else:
        # python2 return unicode(x)        
        return x

In [ ]:
y = [clean_desc(x) for x in descs]

In [ ]:
from striplog import Lexicon
lex = Lexicon.default()

In [ ]:
y = clean_lithology_descriptions(y, lex)

We get a flat list of all the "tokens" but remove stop words ('s', 'the' and the like)

In [ ]:
y = v_lower(y)
vt = v_word_tokenize(y)
flat = np.concatenate(vt)

In [ ]:
import nltk
from nltk.corpus import stopwords

In [ ]:
stoplist = stopwords.words('english')
exclude = stoplist + ['.',',',';',':','(',')','-']
flat = [word for word in flat if word not in exclude]

In [ ]:
len(set(flat))

In [ ]:
df_most_common= token_freq(flat, 50)

In [ ]:
plot_freq(df_most_common)

In [ ]:
df_most_common

## Defining lithology classes and finding primary/secondary lithologies

From the list of most common tokens, we may want to define lithology classes as follows:

In [ ]:
df[LITHO_DESC_COL] = y

In [ ]:
lithologies = ['shale', 'clay','granite','soil','sand', 'porphyry','siltstone','gravel']

In [ ]:
# Prep for visualisation
lithology_color_names = ['lightslategrey', 'olive', 'dimgray', 'chocolate',  'gold',     'tomato', 'teal', 'lavender']


And to capture any of these we devise a regular expression:

In [ ]:
my_lithologies_numclasses = create_numeric_classes(lithologies)

In [ ]:
lithologies_dict = dict([(x,x) for x in lithologies])
lithologies_dict['dacite'] = 'granite'
lithologies_dict['sandstone'] = 'granite'
lithologies_dict['slate'] = 'granite'
lithologies_dict['rock'] = 'granite'
lithologies_dict['ryodacite'] = 'granite'
lithologies_dict['mudstone'] = 'sand' # ??
lithologies_dict['topsoil'] = 'soil' # ??

In [ ]:
any_litho_markers_re = r'shale|clay|granit|soil|sand|porphy|silt|gravel|dacit|slat|rock|stone'
regex = re.compile(any_litho_markers_re)

In [ ]:
lithologies_adjective_dict = {
    'sandy' :  'sand',
    'clayey' :  'clay',
    'clayish' :  'clay',
    'shaley' :  'shale',
    'silty' :  'silt',
    'pebbly' :  'pebble',
    'gravelly' :  'gravel',
    'porphyritic': 'porphyry'
}

In [ ]:
v_tokens = v_word_tokenize(y)
litho_terms_detected = v_find_litho_markers(v_tokens, regex=regex)

Let's see if we detect these lithology markers in each bore log entries  

In [ ]:
zero_mark = [x for x in litho_terms_detected if len(x) == 0 ]
at_least_one_mark = [x for x in litho_terms_detected if len(x) >= 1]
at_least_two_mark = [x for x in litho_terms_detected if len(x) >= 2]
print('There are %s entries with no marker, %s entries with at least one, %s with at least two'%(len(zero_mark),len(at_least_one_mark),len(at_least_two_mark)))

Note: probably need to think of precanned facilities in ela to assess the detection rate in such EDA. Maybe wordcloud not such a bad idea too.

In [ ]:
descs_zero_mark = [y[i] for i in range(len(litho_terms_detected)) if len(litho_terms_detected[i]) == 0 ]

In [ ]:
import random
random.sample(descs_zero_mark,20)
# descs_zero_mark[1:20]

In [ ]:
primary_litho = v_find_primary_lithology(litho_terms_detected, lithologies_dict)

In [ ]:
secondary_litho = v_find_secondary_lithology(litho_terms_detected, primary_litho, lithologies_adjective_dict, lithologies_dict)

In [ ]:
df[PRIMARY_LITHO_COL]=primary_litho
df[SECONDARY_LITHO_COL]=secondary_litho

In [ ]:
df[PRIMARY_LITHO_NUM_COL] = v_to_litho_class_num(primary_litho, my_lithologies_numclasses)
df[SECONDARY_LITHO_NUM_COL] = v_to_litho_class_num(secondary_litho, my_lithologies_numclasses)

In [ ]:
df.head()

## Converting depth below ground to Australian Height Datum elevation

While the bore entries have columns for AHD elevations, many appear to be missing data. Since we have a DEM of the region we can correct this.

In [ ]:
cd = HeightDatumConverter(dem)

In [ ]:
df = cd.add_height(df, 
        depth_from_col=DEPTH_FROM_COL, depth_to_col=DEPTH_TO_COL, 
        depth_from_ahd_col=DEPTH_FROM_AHD_COL, depth_to_ahd_col=DEPTH_TO_AHD_COL, 
        easting_col=EASTING_COL, northing_col=NORTHING_COL, drop_na=False)

In [ ]:
df.head()

In [ ]:
# to be reused in experimental notebooks:
classified_logs_filename = os.path.join(cbr_datadir_out,'classified_logs.pkl')
if not os.path.exists(classified_logs_filename):
    df.to_pickle(classified_logs_filename)

# Viewer

Derived from [voila-gpx-viewer](https://github.com/jtpio/voila-gpx-viewer) and [ipyleaflet docs](https://ipyleaflet.readthedocs.io/en/latest/api_reference/popup.html)

However this is very difficult to adapt, actually use case is quite different from the GPX data. ipyleaflet doc is sparse and how to I guess the content of callbacks??
https://towardsdatascience.com/interactive-controls-for-jupyter-notebooks-f5c94829aee6




In [ ]:
import datetime
import json

from io import StringIO
#from statistics import mean

from bqplot import Axis, Figure, Lines, LinearScale, LogScale
from bqplot.interacts import IndexSelector
from ipyleaflet import basemaps, FullScreenControl, LayerGroup, Map, MeasureControl, Polyline, Marker, CircleMarker, WidgetControl, MarkerCluster
from ipywidgets import Button, HTML, HBox, VBox, Checkbox, FileUpload, Label, Output, IntSlider, Layout, Image, link

In [ ]:
df.head()

In [ ]:
class GlobalThing:
    def __init__(self):
        self.marker_info = dict()
    
    def add_marker_info(self, lat, lon, code):
        self.marker_info[(lat, lon)] = code
    
    def get_code(self, lat, lon):
        return self.marker_info[(lat, lon)]
    
globalthing = GlobalThing()

In [ ]:
out = Output(layout={'border': '1px solid black'})

In [ ]:
def boo():
    out.clear_output()
    with out:
        print("Boo")        

In [ ]:
def data_for_hydrocode(code):
    return df.loc[df['HydroCode'] == code]


In [ ]:
def plot_map(geoloc):
    """
    Plot the GPS trace on a map
    """
    mean_lat = geoloc.Latitude.mean()
    mean_lng = geoloc.Longitude.mean()
    # create the map
    m = Map(center=(mean_lat, mean_lng), zoom=11, basemap=basemaps.Stamen.Terrain)
    # show trace
    markers = []
    def click_handler(**kwargs):
        blah = dict(**kwargs)
        xy = blah['coordinates']
        code = globalthing.get_code(xy[0], xy[1])
        bore_data = df.loc[df['HydroCode'] == code]
        out.clear_output()
        with out:
            print(code)        
            print(bore_data)
    for index, row in geoloc.iterrows():
        message = HTML()
        hc = str(row.HydroCode)
        d = data_for_hydrocode(hc)
        message.value = "HydroCode %s has %s records"%(hc,len(d))
        message.placeholder = "Placeholder"
        message.description = "Description"
        globalthing.add_marker_info(row.Latitude, row.Longitude, row.HydroCode)
        marker = Marker(location=(row.Latitude, row.Longitude))
        marker.on_click(click_handler)
        #marker.popup = message
        markers.append(marker)
    marker_cluster = MarkerCluster(
        markers=markers
    )
    #marker_cluster.on_click(click_handler)
    m.add_layer(marker_cluster);
    m.add_control(FullScreenControl())
    return m

In [ ]:
out

In [ ]:
plot_map(geoloc)

In [ ]:
#globalthing.marker_info

In [ ]:
click_handler(coordinates = [-35.33895682, 149.26627207])


## Interpolate over a regular grid


In [ ]:
# max/min bounds
shp_bbox = get_bbox(bore_locations)
shp_bbox

In [ ]:
raster_bbox = dem.bounds
raster_bbox

In [ ]:
x_min = max(shp_bbox[0], raster_bbox[0])
x_max = min(shp_bbox[2], raster_bbox[2])
y_min = max(shp_bbox[1], raster_bbox[1])
y_max = min(shp_bbox[3], raster_bbox[3])


In [ ]:
grid_res = 200
m = create_meshgrid_cartesian(x_min, x_max, y_min, y_max, grid_res)

In [ ]:
[x.shape for x in m]

In [ ]:
dem_array = surface_array(dem, x_min, y_min, x_max, y_max, grid_res)

In [ ]:
dem_array.shape

In [ ]:
dem_array[dem_array <= 0.0] = np.nan

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
plt.imshow(to_carto(dem_array), cmap='terrain')

In [ ]:
dem_array_data = {'bounds': (x_min, x_max, y_min, y_max), 'grid_res': grid_res, 'mesh_xy': m, 'dem_array': dem_array}

In [ ]:
import pickle

fp = os.path.join(cbr_datadir_out, 'dem_array_data.pkl')
if not os.path.exists(fp):
    with open(fp, 'wb') as handle:
        pickle.dump(dem_array_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

We need to define min and max heights on the Z axis for which we interoplate. We use the KNN algorithm with 10 neighbours. We should use a domain such that there are enough points for each height. Let's find visually heights with at least 10 records

In [ ]:
n_bins=100
p = df.hist(column=[DEPTH_FROM_AHD_COL,DEPTH_TO_AHD_COL], sharex=True, sharey=True, bins=n_bins, figsize=(15,5))
for axes in p:
    axes[0].set_yscale("log", nonposy='clip')

In [ ]:
n_neighbours=10
ahd_min=530
ahd_max=710

z_ahd_coords = np.arange(ahd_min,ahd_max,1)
dim_x,dim_y = m[0].shape
dim_z = len(z_ahd_coords)
dims = (dim_x,dim_y,dim_z)

In [ ]:
dims

In [ ]:
lithology_3d_array=np.empty(dims)

In [ ]:
gi = GridInterpolation(easting_col=EASTING_COL, northing_col=NORTHING_COL)

In [ ]:
gi.interpolate_volume(lithology_3d_array, df, PRIMARY_LITHO_NUM_COL, z_ahd_coords, n_neighbours, m)

In [ ]:
# Burn DEM into grid
z_index_for_ahd = z_index_for_ahd_functor(b=-ahd_min)

In [ ]:
dem_array.shape, m[0].shape, lithology_3d_array.shape

In [ ]:
burn_volume(lithology_3d_array, dem_array, z_index_for_ahd, below=False)

In [ ]:
# to be reused in experimental notebooks:
interp_litho_filename = os.path.join(cbr_datadir_out,'3d_primary_litho.pkl')
if not os.path.exists(interp_litho_filename):
    with open(interp_litho_filename, 'wb') as handle:
        pickle.dump(lithology_3d_array, handle, protocol=pickle.HIGHEST_PROTOCOL)

## 2D visualisations

In [ ]:
lithology_cmap = discrete_classes_colormap(lithology_color_names) # Later for exporting to RGB geotiffs??
litho_legend_display_info = [(lithology_cmap[i], lithologies[i], lithology_color_names[i]) for i in range(len(lithologies))]

In [ ]:
litho_legend = legend_fig(litho_legend_display_info)

In [ ]:
cms = cartopy_color_settings(lithology_color_names)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
imgplot = plt.imshow(to_carto(lithology_3d_array[:, :, z_index_for_ahd(540)]), cmap=cms['cmap'])
title = plt.title('Primary litho at +540mAHD')

## 3D visualisation

In [ ]:
from ela.visual3d import *

In [ ]:
xx, yy = dem_array_data['mesh_xy']

In [ ]:
from mayavi import mlab

In [ ]:
vis_litho = LithologiesClassesVisual3d(lithologies, lithology_color_names, 'black')

In [ ]:
# TODO: problematic with this data - investigate
# vis_litho.render_classes_planar(lithology_3d_array, 'Primary lithology')

In [ ]:
# vis_litho.render_class(lithology_3d_array, 0)

ela has facilities to visualise overlaid information: DEM, classified bore logs, and volumes of interpolated lithologies. This is important to convey .

First a bit of data filling for visual purposes, as NaN lithology class codes may cause issues.

In [ ]:
df_infilled = df.fillna({PRIMARY_LITHO_NUM_COL: -1.0})
df_infilled = df_infilled[(df_infilled[DEPTH_TO_AHD_COL] > (ahd_min-20))]

In [ ]:
# A factor to apply to Z coordinates, otherwise things would be squashed visually along the heights.
# Would prefer a visual only scaling factor, but could not find a way to do so. 
Z_SCALING = 20.0

In [ ]:
z_coords = np.arange(ahd_min,ahd_max,1)

In [ ]:
overlay_vis_litho = LithologiesClassesOverlayVisual3d(lithologies, lithology_color_names, 'black', dem_array_data, z_coords, Z_SCALING, df_infilled, PRIMARY_LITHO_NUM_COL)

In [ ]:
def view_class(value):
    f = overlay_vis_litho.view_overlay(value, lithology_3d_array)
    return f

In [ ]:
f = view_class(7.0)

In [ ]:
f = view_class(1.0)

In [ ]:
f = view_class(2.0)

![3D Interpolated overlay primary lithology quartz](img/snapshot_quartz.png)

In [ ]:
vis_litho = LithologiesClassesVisual3d(lithologies, lithology_color_names, 'black')

In [ ]:
vis_litho.render_classes_planar(lithology_3d_array, 'Primary lithology')

### Export volumes for depth from ground

In [ ]:
max_depth = 100
dim_x,dim_y = (lithology_3d_array.shape[0],lithology_3d_array.shape[1])
dim_z = max_depth + 1
dims = (dim_x,dim_y,dim_z)

In [ ]:
grid_res = dem_array_data['grid_res']
x_min, x_max, y_min, y_max = dem_array_data['bounds']
xx, yy = dem_array_data['mesh_xy']
dem_array = dem_array_data['dem_array']

In [ ]:
litho_classes_depth=np.empty(dims)
for depth in range(0, dim_z, 1):
    s = slice_volume(lithology_3d_array, dem_array - depth, z_index_for_ahd)
    litho_classes_depth[:,:,(max_depth - depth)] = s

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
imgplot = plt.imshow(to_carto(litho_classes_depth[:,:,dim_z-30]), cmap=cms['cmap'])
t = plt.title('Primary lithologies at 30m depth')

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
imgplot = plt.imshow(to_carto(litho_classes_depth[:,:,dim_z-1]), cmap=cms['cmap'])
t = plt.title('Primary lithologies at 1m depth')

In [ ]:
def plot_borehole_data(df):
    # We may have something fancy visual down the track, for now, a dataframe subsetting. 
    px = [p.Time for p in points]
    py = [p.CND_011 for p in points]

    x_scale, y_scale = LinearScale(), LogScale()
    x_scale.allow_padding = False
    x_ax = Axis(label='Time (s)', scale=x_scale)
    y_ax = Axis(label='CND 011(?)', scale=y_scale, orientation='vertical')

    lines = Lines(x=px, y=py, scales={'x': x_scale, 'y': y_scale})

    elevation = Figure(title='CND 011 Chart', axes=[x_ax, y_ax], marks=[lines])
    elevation.layout.width = 'auto'
    elevation.layout.height = 'auto'
    elevation.layout.min_height = '500px'

    elevation.interaction = IndexSelector(scale=x_scale)

    return elevation

In [ ]:
def link_geo_borehole(geomap, boreholelayer):
    """
    Links the geolocation of the markers to the display of the bvorehole log
    Changing the selection on the marker will update the
    borehole display
    """
    # add a checkbox to auto center
    autocenter = Checkbox(value=False, description='Auto Center')
    autocenter_control = WidgetControl(widget=autocenter, position='bottomright')
    geomap.add_control(autocenter_control)

    brushintsel = geomap.interaction
    def update_range(change):
        """
        Update the position on the map when the elevation
        graph selector changes
        """
        if brushintsel.selected.shape != (1,):
            return
        marker.visible = True
        selected = brushintsel.selected # time stamp in seconds for a day
        point = find_point(selected)
        marker.location = (point.Latitude, point.Longitude)
        if autocenter.value:
            trace.center = marker.location
        #position = max(0, int((selected / distance_from_start) * len(points)))
    brushintsel.observe(update_range, 'selected')

    
def link_trace_elevation(trace, elevation, points):
    """
    Link the trace the elevation graph.
    Changing the selection on the elevation will update the
    marker on the map
    """
    times = np.asarray([p.Time for p in points])

    def find_point(time):
        """
        Find a point given the time
        """
        dist_1 = abs(times - time)
        pos = np.argmin(dist_1)
        return points[pos]
    
    # add a checkbox to auto center
    autocenter = Checkbox(value=False, description='Auto Center')
    autocenter_control = WidgetControl(widget=autocenter, position='bottomright')
    trace.add_control(autocenter_control)
    # mark the current position on the map
    start = points[0]
    marker = CircleMarker(visible=False, location=(start.Latitude, start.Longitude),
                          radius=10, color="green", fill_color="green")
    trace.add_layer(marker)
    brushintsel = elevation.interaction
    def update_range(change):
        """
        Update the position on the map when the elevation
        graph selector changes
        """
        if brushintsel.selected.shape != (1,):
            return
        marker.visible = True
        selected = brushintsel.selected # time stamp in seconds for a day
        point = find_point(selected)
        marker.location = (point.Latitude, point.Longitude)
        if autocenter.value:
            trace.center = marker.location
        #position = max(0, int((selected / distance_from_start) * len(points)))
    brushintsel.observe(update_range, 'selected')


In [ ]:
from ipywidgets import FloatSlider
interact(slow_function,i=FloatSlider(min=1e5, max=1e7, step=1e5));

In [ ]:
def plot_gpx(points):
    trace = plot_map(points)
    elevation = plot_elevation(points)
    debug = Label(value='')
    display(trace)
    display(elevation)
    display(debug)
    link_trace_elevation(trace, elevation, points)

In [ ]:
plot_gpx(points)